In [40]:
import platform
import sklearn
import pandas as pd
import numpy as np
import os
from __future__ import division
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score

In [41]:
season_data = ("nba_season_data.csv")

In [42]:
from sklearn.metrics import f1_score, make_scorer, classification_report
scorer = make_scorer(f1_score, pos_label=None, average='weighted')

In [43]:
results = pd.read_csv(season_data, parse_dates=["Date"])
results.rename(columns = {'Visitor/Neutral':'Visitor', 'PTS':'VisitorPts', 'Home/Neutral':'Home', 'PTS.1':'HomePts', 'Unnamed:6':'Score Type', 'Unnamed: 7':'OT?'}, inplace = True)
results["VisitorPts"] = results["VisitorPts"].apply(pd.to_numeric, errors='coerce')
results["HomePts"] = results["HomePts"].apply(pd.to_numeric, errors='coerce')
results["Home Team Win"] = results["HomePts"] > results["VisitorPts"]

In [44]:
num_games = results["Home Team Win"].count()
num_home_wins = results["Home Team Win"].sum()
home_win_percentage =  (num_home_wins/num_games * 100)

In [45]:
#CLASSIFICATION BASED ON LAST GAMES FOR TEAMS
from collections import defaultdict
results["Home Last Win"] = False
results["Visitor Last Win"] = False
won_last = defaultdict(int)

for i, r in results.iterrows():
    home_team = r["Home"]
    visitor_team = r["Visitor"]
    r["Home Last Win"] = won_last[home_team]
    r["Visitor Last Win"] = won_last[visitor_team]
    results.ix[i] = r
    won_last[home_team] = r["Home Last Win"]
    won_last[visitor_team] = not r["Home Last Win"]

clf = DecisionTreeClassifier(random_state=14)
x_previouswins = results[['Home Last Win', 'Visitor Last Win']].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, x_previouswins, results["Home Team Win"].values, scoring=scorer)
print("Using just the last result from the home and visitor teams")
print("F1: {0:.4f}".format(np.mean(scores)))

/Users/TanmayGhai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if sys.path[0] == '':


Using just the last result from the home and visitor teams
F1: 0.4269


In [46]:
#CLASSIFICATION BASED ON WIN STREAKS FOR TEAMS
from collections import defaultdict
results["Home Win Streak"] = 0
results["Visitor Win Streak"] = 0
win_streak = defaultdict(int)

for i, r in results.iterrows():
    home_team = r["Home"]
    visitor_team = r["Visitor"]
    r["Home Win Streak"] = win_streak[home_team]
    r["Home Win Streak"] = win_streak[visitor_team]
    results.ix[i] = r
    if r["Home Team Win"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1 

clf = DecisionTreeClassifier(random_state=14)
x_previouswins = results[['Home Last Win', 'Visitor Last Win', "Home Win Streak", "Visitor Win Streak"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, x_previouswins, results["Home Team Win"].values, scoring=scorer)
print("Using whether the home team has a win streak")
print("F1: {0:.4f}".format(np.mean(scores)))

Using whether the home team has a win streak
F1: 0.5291


In [134]:
standings_data = ("nba_standings_data.csv")
standings = pd.read_csv(standings_data, skiprows=[0])
def home_ranked_higher(r):
    home_team = r["Home"]
    home_team_index = standings[standings["Team"] == home_team].index[0]
    visitor_team = r["Visitor"]
    visitor_team_index = standings[standings["Team"] == visitor_team].index[0]
    home_rank = standings.loc[home_team_index]["Rk"]
    visitor_rank = standings.loc[visitor_team_index]["Rk"]
    return home_rank < visitor_rank
#results["Home Team Ranked Higher"] = results.apply(home_ranked_higher, axis=1)
standings[standings["Team"] == 1].index

Int64Index([], dtype='int64')